# Parse KnotInfo polynomials
notebook to extract vectors of coefficients from data downloaded from https://knotinfo.math.indiana.edu/

In [ ]:
import numpy as np
import pandas as pd
import cypari2

from tqdm.auto import tqdm

## utils

In [ ]:
pari = cypari2.Pari()

In [ ]:
def min_max_degree(list_of_poly, variables):
    min_max = {}
    for v in variables:
        min_max[v] = {}
        min_max[v]["max"] = -1000
        min_max[v]["min"] = +1000

    for poly in tqdm(list_of_poly):

        pari_poly = pari(poly)

        for v in variables:
            # max degree
            degree = pari.poldegree(pari_poly, v)
            if degree > min_max[v]["max"]:
                min_max[v]["max"] = degree

            # min degree
            # convert v to 1/v and get the max degree
            pari_poly = pari.subst(pari_poly, v, "1/{}".format(v))
            neg_degree = -1 * pari.poldegree(pari_poly, v)
            if neg_degree < min_max[v]["min"]:
                min_max[v]["min"] = neg_degree

    return min_max


def create_coeff_vector(poly, min_t, max_t, symbol="t"):
    off_t = -min_t

    coeff = np.zeros(max_t - min_t + 1, dtype=int)

    for idx_t in range(min_t, max_t + 1):
        coeff[idx_t + off_t] = pari.polcoef(poly, idx_t, symbol)

    return coeff


def create_colnames(min_t, max_t, symbol="t"):
    colnames = []
    for idx_t in range(min_t, max_t + 1):
        colnames.append(symbol + str(idx_t))
    return colnames


def create_coeff_vector_2d(poly, min_1, max_1, min_2, max_2, symbol_1, symbol_2):
    off_q = -min_1
    off_t = -min_2

    coeff = np.zeros((max_1 - min_1 + 1, max_2 - min_2 + 1), dtype=int)

    for idx_1 in range(min_1, max_1 + 1):
        for idx_2 in range(min_2, max_2 + 1):
            coeff[idx_1 + off_q, idx_2 + off_t] = pari.polcoef(
                pari.polcoef(poly, idx_2, symbol_2), idx_1, symbol_1
            )

    return coeff.reshape(-1)


def create_colnames_2d(min_1, max_1, min_2, max_2, symbol_1, symbol_2):
    colnames = []
    for idx_1 in range(min_1, max_1 + 1):
        for idx_2 in range(min_2, max_2 + 1):
            colnames.append(symbol_1 + str(idx_1) + "_" + symbol_2 + str(idx_2))
    return colnames


def create_coeff_vector_3d(
    poly, min_1, max_1, min_2, max_2, min_3, max_3, symbol_1, symbol_2, symbol_3
):
    off_1 = -min_1
    off_2 = -min_2
    off_3 = -min_3

    coeff = np.zeros(
        (max_1 - min_1 + 1, max_2 - min_2 + 1, max_3 - min_3 + 1), dtype=int
    )

    for idx_1 in range(min_1, max_1 + 1):
        for idx_2 in range(min_2, max_2 + 1):
            for idx_3 in range(min_3, max_3 + 1):
                coeff[idx_1 + off_1, idx_2 + off_2, idx_3 + off_3] = pari.polcoef(
                    pari.polcoef(pari.polcoef(poly, idx_3, symbol_3), idx_2, symbol_2),
                    idx_1,
                    symbol_1,
                )

    return coeff.reshape(-1)


def create_colnames_3d(
    min_1, max_1, min_2, max_2, min_3, max_3, symbol_1, symbol_2, symbol_3
):
    colnames = []
    for idx_1 in range(min_1, max_1 + 1):
        for idx_2 in range(min_2, max_2 + 1):
            for idx_3 in range(min_3, max_3 + 1):
                colnames.append(
                    symbol_1
                    + str(idx_1)
                    + "_"
                    + symbol_2
                    + str(idx_2)
                    + "_"
                    + symbol_3
                    + str(idx_3)
                )
    return colnames

## load data

In [ ]:
knotinfo_df = pd.read_csv("data/knotinfo_polys.csv")
knotinfo_df

## Alexander

In [ ]:
list_of_poly = [pari(p) for p in knotinfo_df.Alexander.tolist()]

In [ ]:
min_max = min_max_degree(list_of_poly, variables=["t"])
print(min_max)
min_t = min_max["t"]["min"]
max_t = min_max["t"]["max"]

In [ ]:
p_coeff = [create_coeff_vector(p, min_t, max_t) for p in tqdm(list_of_poly)]

In [ ]:
coeff_names = create_colnames(min_t, max_t)

coeff_df = pd.DataFrame(data=p_coeff, columns=coeff_names)

coeff_df["knot_id"] = knotinfo_df.Name

coeff_df = coeff_df[
    [
        "knot_id",
    ]
    + coeff_names
]

In [ ]:
coeff_df

In [ ]:
coeff_df.to_csv("data/alex_knotinfo.csv", index=False)

## Jones

In [ ]:
list_of_poly = [pari(p) for p in knotinfo_df.Jones.tolist()]

min_max = min_max_degree(list_of_poly, variables=["t"])
print(min_max)
min_t = min_max["t"]["min"]
max_t = min_max["t"]["max"]

p_coeff = [create_coeff_vector(p, min_t, max_t) for p in tqdm(list_of_poly)]

coeff_names = create_colnames(min_t, max_t)

coeff_df = pd.DataFrame(data=p_coeff, columns=coeff_names)

coeff_df["knot_id"] = knotinfo_df.Name

coeff_df = coeff_df[
    [
        "knot_id",
    ]
    + coeff_names
]

In [ ]:
coeff_df

In [ ]:
coeff_df.to_csv("data/jones_knotinfo.csv", index=False)

## homflypt

In [ ]:
homfly = [poly for poly in tqdm(knotinfo_df["HOMFLY"].to_numpy())]
homfly[0]

In [ ]:
min_max = min_max_degree(homfly, variables=["v", "z"])

max_v = min_max["v"]["max"]
min_v = min_max["v"]["min"]

max_z = min_max["z"]["max"]
min_z = min_max["z"]["min"]

min_max

In [ ]:
c_list = []

for p in tqdm(homfly):
    p_coeff = create_coeff_vector_2d(
        pari(p), min_v, max_v, min_z, max_z, symbol_1="v", symbol_2="z"
    )
    c_list.append(p_coeff)

In [ ]:
coeff_names = create_colnames_2d(min_v, max_v, min_z, max_z, symbol_1="v", symbol_2="z")

coeff_df = pd.DataFrame(data=c_list, columns=coeff_names)
# add info
coeff_df["knot_id"] = knotinfo_df.Name

coeff_df = coeff_df[["knot_id"] + coeff_names]
coeff_df.head()

In [ ]:
coeff_df.to_csv("data/homfly_knotinfo.csv", index=False)

## HFK

In [ ]:
## pari does not like tha string, lets' fix it
pol = (
    knotinfo_df["HFK Polyomial"]
    .to_numpy()[0]
    .replace("  ", "")
    .replace(" + ", "+")
    # .replace(" ", "*")
    .replace("a", "*a")
    .replace("m", "*m")
)
pol

In [ ]:
pari(pol)

In [ ]:
hfk = [
    poly.replace("  ", "").replace(" + ", "+").replace("a", "*a").replace("m", "*m")
    for poly in tqdm(knotinfo_df["HFK Polyomial"].to_numpy())
]
hfk[0]

In [ ]:
min_max = min_max_degree(hfk, variables=["a", "m"])
min_max

In [ ]:
max_a = min_max["a"]["max"]
min_a = min_max["a"]["min"]

max_m = min_max["m"]["max"]
min_m = min_max["m"]["min"]


print(
    "a",
    min_a,
    max_a,
)
print(
    "m",
    min_m,
    max_m,
)

In [ ]:
c_list = []

for p in tqdm(hfk):
    p_coeff = create_coeff_vector_2d(
        pari(p), min_a, max_a, min_m, max_m, symbol_1="a", symbol_2="m"
    )
    c_list.append(p_coeff)

In [ ]:
coeff_names = create_colnames_2d(min_a, max_a, min_m, max_m, symbol_1="a", symbol_2="m")

coeff_df = pd.DataFrame(data=c_list, columns=coeff_names)
# add info
coeff_df["knot_id"] = knotinfo_df.Name

coeff_df = coeff_df[["knot_id"] + coeff_names]
coeff_df.head()

In [ ]:
coeff_df.to_csv("data/HFK_knotinfo.csv", index=False)

## KH Odd Red Z Poly

In [ ]:
## pari does not like tha string, lets' fix it
pol = (
    knotinfo_df["KH Odd Red Z Poly"]
    .to_numpy()[0]
    .replace("  ", "")
    .replace(" + ", "+")
    .replace(" ", "*")
)
pol

In [ ]:
pari(pol)

In [ ]:
khov = [
    poly.replace("  ", "").replace(" + ", "+").replace(" ", "*")
    for poly in tqdm(knotinfo_df["KH Odd Red Z Poly"].to_numpy())
]
khov[0]

In [ ]:
min_max = min_max_degree(khov, variables=["q", "t", "T"])
min_max

In [ ]:
max_q = min_max["q"]["max"]
min_q = min_max["q"]["min"]

max_t = min_max["t"]["max"]
min_t = min_max["t"]["min"]

max_T = min_max["T"]["max"]
min_T = min_max["T"]["min"]

In [ ]:
c_list = []

for p in tqdm(khov):
    p_coeff = create_coeff_vector_3d(
        pari(p),
        min_q,
        max_q,
        min_t,
        max_t,
        min_T,
        max_T,
        symbol_1="q",
        symbol_2="t",
        symbol_3="T",
    )
    c_list.append(p_coeff)

In [ ]:
pari(p)

In [ ]:
coeff_names = create_colnames_3d(
    min_q,
    max_q,
    min_t,
    max_t,
    min_T,
    max_T,
    symbol_1="q",
    symbol_2="t",
    symbol_3="T",
)

coeff_df = pd.DataFrame(data=c_list, columns=coeff_names)
# add info
coeff_df["knot_id"] = knotinfo_df.Name

coeff_df = coeff_df[["knot_id"] + coeff_names]
coeff_df.head()

In [ ]:
coeff_df.to_csv("data/khovanov_knotinfo.csv", index=False)